In [1]:
# Install Gradio
!pip install gradio --quiet

import gradio as gr
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Create synthetic dataset
np.random.seed(42)
data_size = 500

ages = np.random.randint(18, 40, size=data_size)
education_levels = np.random.choice(['High School', 'Bachelors', 'Masters', 'PhD'], size=data_size)
experience_years = np.random.randint(0, 15, size=data_size)
scores = np.round(np.random.uniform(5.0, 10.0, size=data_size), 2)

# Simple rule: More experience, higher education, and better score → more likely to be accepted
accepted = ((experience_years > 2) &
            (scores > 7.0) &
            ((education_levels == 'Masters') | (education_levels == 'PhD'))).astype(int)

# Convert to DataFrame
df = pd.DataFrame({
    'Age': ages,
    'Education': education_levels,
    'Experience': experience_years,
    'Score': scores,
    'Accepted': accepted
})

# One-hot encode education
df_encoded = pd.get_dummies(df, columns=['Education'])

# Features and target
X = df_encoded.drop('Accepted', axis=1)
y = df_encoded['Accepted']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Accuracy
acc = accuracy_score(y_test, model.predict(X_test))
print(f"Model accuracy: {acc*100:.2f}%")

# Store categories for inference
education_categories = ['Education_Bachelors', 'Education_High School', 'Education_Masters', 'Education_PhD']

# Prediction function
def predict_acceptance(age, education, experience, score):
    input_dict = {
        'Age': age,
        'Experience': experience,
        'Score': score,
    }

    # One-hot encode education
    for cat in education_categories:
        input_dict[cat] = 1 if cat == f"Education_{education}" else 0

    input_df = pd.DataFrame([input_dict])
    pred = model.predict(input_df)[0]
    return "✅ Application Accepted" if pred == 1 else "❌ Application Rejected"

# Gradio UI
demo = gr.Interface(
    fn=predict_acceptance,
    inputs=[
        gr.Number(label="Age"),
        gr.Dropdown(choices=['High School', 'Bachelors', 'Masters', 'PhD'], label="Education Level"),
        gr.Number(label="Years of Experience"),
        gr.Number(label="CGPA / Score (out of 10)")
    ],
    outputs="text",
    title="📄 Application Acceptor (Decision Tree)",
    description="Predicts whether an application will be accepted based on profile info."
)

demo.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.6 MB/s eta 0:00:00
Model accuracy: 100.00%
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b1f72ac866bd4f758a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal i